In [1]:
import numpy as np
import tensorflow as tf
wordsList=np.load('wordsList.npy') #包含40万个词的python列表
wordsList=np.load('wordsList.npy') #包含40万个词的python列表
wordsList=wordsList.tolist()
wordsList=[word.decode('UTF-8') for word in wordsList]
wordVectors=np.load('wordVectors.npy')
baseballIndex=wordsList.index('baseball')
print(wordVectors[baseballIndex])
import pandas as pd
#读入数据
df=pd.read_csv('labeledTrainData.tsv',sep='\t',escapechar='\\')

numDimensions=300
print("down")

maxSeqLength=250
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
#清洗数据，HTML字符使用空格替代
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

# #生成索引矩阵，得到24500*250的索引矩阵
# ids=np.zeros((24500,maxSeqLength),dtype='int32')
# #print(ids.shape) #输出结果为（24500,250）

# fileCounter=0
# for pf in range(0,len(df)): 
#     #print(pf)
#     indexCounter=0
#     cleanedLine=cleanSentences(df['review'][pf])
#     split=cleanedLine.split()
#     for word in split:
#         try:
#             #print('111')
#             ids[fileCounter][indexCounter]=wordsList.index(word)
#         except ValueError:
#             ids[fileCounter][indexCounter]=399999
#         indexCounter=indexCounter+1
#         if indexCounter>=maxSeqLength:
#             break        
#     fileCounter=fileCounter+1 
# print('down1')
# np.save('C:/NLP/idsMatrix',ids)
#上述注释后，将生成的索引矩阵保存到idsMatrix.npy文件中。避免了每次都要生成索引矩阵
ids=np.load('idsMatrix.npy')
print(ids.shape)

#辅助函数
from random import randint
def getTrainBatch():
    labels=[]
    arr=np.zeros([batchSize,maxSeqLength])
    i=0
    for i in range(0,32):
        j=0
        while j<1:
            num=randint(1,19600)
            if df['sentiment'][num-1]==1:
                j=j+1
                labels.append([1,0])
                arr[2*i]=ids[num-1:num]
        j=0
        while j<1:
            num=randint(1,19600)
            if df['sentiment'][num-1]==0:
                j=j+1
                labels.append([0,1])
                arr[2*i+1]=ids[num-1:num]
    return arr,labels
print('down')


batchSize=64 #批处理大小
lstmUnits=64 #LSTM单元个数
numClasses=2 #分类类别
iterations=50000 #训练次数
print('down')

[-1.9327    1.0421   -0.78515   0.91033   0.22711  -0.62158  -1.6493
  0.07686  -0.5868    0.058831  0.35628   0.68916  -0.50598   0.70473
  1.2664   -0.40031  -0.020687  0.80863  -0.90566  -0.074054 -0.87675
 -0.6291   -0.12685   0.11524  -0.55685  -1.6826   -0.26291   0.22632
  0.713    -1.0828    2.1231    0.49869   0.066711 -0.48226  -0.17897
  0.47699   0.16384   0.16537  -0.11506  -0.15962  -0.94926  -0.42833
 -0.59457   1.3566   -0.27506   0.19918  -0.36008   0.55667  -0.70315
  0.17157 ]
down
(24500, 250)
down
down


In [8]:
import tensorflow.compat.v1 as tf

tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [None, numClasses])
input_data = tf.placeholder(tf.int32, [None, maxSeqLength])
data = tf.Variable(
    tf.zeros([batchSize, maxSeqLength, numDimensions]), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors, input_data)

# lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits)
#lstmCell = tf.contrib.rnn.LSTMCell(lstm_units)
# lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
drop = tf.nn.rnn_cell.DropoutWrapper(lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

pre=tf.nn.softmax(prediction)
print(pre)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

print('down')


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/tmp/ipykernel_800322/1376142830.py:12: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstmCell = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits)
/usr/local/lib/python3.8/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/usr/local/lib/python3.8/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:757: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/GatherV2_grad/Resha

Tensor("Softmax:0", shape=(?, 2), dtype=float32)
down


In [10]:
sess=tf.InteractiveSession()
saver=tf.train.Saver()
sess.run(tf.global_variables_initializer())
for i in range(0,40001):
    nextBatch,nextBatchLabels=getTrainBatch();
    #print(nextBatch.shape)
    #print(nextBatchLabels)
    sess.run(optimizer,{input_data:nextBatch,labels:nextBatchLabels})
    if(i%1000==0 and i!=0):
        loss_=sess.run(loss,{input_data:nextBatch,labels:nextBatchLabels})
        accuracy_=sess.run(accuracy,{input_data:nextBatch,labels:nextBatchLabels})
        print("i...{}".format(i),"loss... {}".format(loss_),"accuracy {}".format(accuracy_))
    if(i%10000==0 and i!=0):
        save_path=saver.save(sess,"/home/id-none/myfile/models1/pretrained_lstm.ckpt",global_step=i)
        print("saved to %s"%save_path)
print('down')

2022-01-20 03:22:20.514939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:38:00.0, compute capability: 8.6
2022-01-20 03:22:20.515507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6
/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1771: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


i...1000 loss... 0.6392914056777954 accuracy 0.609375
i...2000 loss... 0.6784121990203857 accuracy 0.546875
i...3000 loss... 0.7337830066680908 accuracy 0.484375
i...4000 loss... 0.41816359758377075 accuracy 0.796875
i...5000 loss... 0.2896137535572052 accuracy 0.890625
i...6000 loss... 0.2938542664051056 accuracy 0.875
i...7000 loss... 0.2388550490140915 accuracy 0.9375
i...8000 loss... 0.2884316146373749 accuracy 0.9375
i...9000 loss... 0.2565196752548218 accuracy 0.90625
i...10000 loss... 0.20505045354366302 accuracy 0.890625
saved to /home/id-none/myfile/models1/pretrained_lstm.ckpt-10000
i...11000 loss... 0.21898305416107178 accuracy 0.921875
i...12000 loss... 0.20707285404205322 accuracy 0.953125
i...13000 loss... 0.06143299490213394 accuracy 0.984375
i...14000 loss... 0.057599231600761414 accuracy 0.96875
i...15000 loss... 0.14526578783988953 accuracy 0.984375
i...16000 loss... 0.035033874213695526 accuracy 1.0
i...17000 loss... 0.007881193421781063 accuracy 1.0
i...18000 loss..

In [11]:
# sess=tf.InteractiveSession()
# saver=tf.train.Saver()
# saver.restore(sess,"C:/NLP/models1/pretrained_lstm.ckpt-20000")
# print('down')
# from sklearn.model_selection import train_test_split
# in2 = np.load('C:/NLP//train_review.npy')
# out2 = np.load('C:/NLP//train_lable.npy')
# print(in2.shape) #输出（24500，250）
# print(out2.shape)#输出（24500，2）

# x_test=in2
# y_test=out2
# #print(X_train.shape)
# #print(Y_train[1])
# print(x_test.shape)
# print(y_test.shape)
# arr=np.zeros([1,maxSeqLength])
# print(arr.shape)
# sum=0
# for i in range(0,1000):
#     labels1 = np.array([y_test[i]])
#     arr = np.array([x_test[i]])
#     #print(arr.shape)
#     #print(labels1)
#     #print("accuracy for test:",(sess.run(accuracy,{input_data:arr,labels:labels1})))
#     #print("labels1:",labels1,"....result for test:",(sess.run(pre,{input_data:arr,labels:labels1})),"accuracy for test:",(sess.run(accuracy,{input_data:arr,labels:labels1})))
#     sum=sum+sess.run(accuracy,{input_data:arr,labels:labels1})
# print(sum)
print('down')

down


In [11]:
import numpy as np
test = np.load('test_review.npy')
test

array([[1.00000e+02, 1.46630e+04, 3.60000e+01, ..., 1.00000e+01,
        7.70000e+01, 1.24700e+03],
       [4.10000e+01, 3.97800e+03, 2.01534e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.17000e+02, 2.01534e+05, 1.06900e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [4.10000e+01, 8.35000e+02, 9.66800e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.60000e+01, 3.55000e+02, 1.90000e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [4.10000e+01, 1.14000e+03, 1.17000e+02, ..., 1.03000e+02,
        1.56600e+03, 3.70000e+01]])

In [43]:
sess=tf.InteractiveSession()
saver=tf.train.Saver()
saver.restore(sess,"/home/id-none/myfile/models1/pretrained_lstm.ckpt-40000")
print('down')

#预测
in3 = np.load('test.npy')


x_test=in3

print(x_test.shape)

arr=np.zeros([1,maxSeqLength])
print(arr.shape)
numpy_data=[]
sum=0
for i in range(0,2):
    arr = np.array([x_test[i]])
    print("result for test:",(sess.run(pre,{input_data:arr})))
    numpy_data.append(sess.run(pre,{input_data:arr}))
    #print("labels1:",labels1,"....result for test:",(sess.run(pre,{input_data:arr,labels:labels1})),"accuracy for test:",(sess.run(accuracy,{input_data:arr,labels:labels1})))
    #sum=sum+sess.run(accuracy,{input_data:arr,labels:labels1})
#print(sum)
print('down')

INFO:tensorflow:Restoring parameters from /home/id-none/myfile/models1/pretrained_lstm.ckpt-40000


2022-01-20 07:22:22.174885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:38:00.0, compute capability: 8.6
2022-01-20 07:22:22.175400: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


down
(22000, 250)
(1, 250)
result for test: [[0.85664546 0.14335456]]
result for test: [[0.9939797  0.00602033]]
down


In [48]:
print(numpy_data[0])
print(numpy_data[0][0])
print(numpy_data[0][0][1])
numpy_data

[[0.85664546 0.14335456]]
[0.85664546 0.14335456]
0.14335456


[array([[0.85664546, 0.14335456]], dtype=float32),
 array([[0.9939797 , 0.00602033]], dtype=float32)]

In [49]:
result_data=[]
print(in3.shape)
for i in range(0,22000):
    arr = np.array([x_test[i]])
    result_data.append(sess.run(pre,{input_data:arr}))
print('down')

(22000, 250)
down


In [40]:
print(result_data[0])

[[0.85664546 0.14335456]]


In [41]:
print(result_data[1])
print(numpy_data[1][0])

[[0.9939797  0.00602033]]
[0.9939797  0.00602033]


In [51]:
f=open('possubmission.txt','w')
for i in range (22000):
    #print(i)
    f.write('\n'+str(result_data[i][0][0]))
f.close()
print('down')

down


In [35]:
g=open('negsubmission.txt','w')
for i in range (22000):
    #print(i)
    g.write('\n'+str(result_data[i][0][1]))
g.close()
print('down')

down
